In [1]:
import music_trees as mt
import glob

# let's look at which results we have available
result_csvs = glob.glob(str(mt.RESULTS_DIR / 'taxonomies'/ '*.csv'))
result_csvs

Global seed set to 42
/home/hugo/venv/lib/python3.7/site-packages/medleydb/__init__.py:64: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/home/hugo/venv/lib/python3.7/site-packages/medleydb/__init__.py:72: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


['/home/hugo/lab/music-trees/results/taxonomies/random-taxonomy-v0.csv',
 '/home/hugo/lab/music-trees/results/taxonomies/dmdb-h2protonet-v1.csv',
 '/home/hugo/lab/music-trees/results/taxonomies/dmdb-baseline-v0.csv',
 '/home/hugo/lab/music-trees/results/taxonomies/origin-taxonomy.csv']

In [2]:
import pandas as pd
dfs = [pd.read_csv(p) for p in result_csvs]
df = pd.concat(dfs, ignore_index=True)
df

,Unnamed: 0,episode_idx,metric,value,tag,model,n_shot,n_class,logger,checkpoint_callback,...,dataset,batch_size,num_workers,n_query,model_name,learning_rate,d_root,height,taxonomy_name,loss_decay
0,0,0,f1_micro,0.793519,protonet,random-taxonomy_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,1,random-taxonomy,-0.5
1,1,0,f1_macro,0.796673,protonet,random-taxonomy_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,1,random-taxonomy,-0.5
2,2,0,hlca-mistake,2.769058,protonet,random-taxonomy_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,1,random-taxonomy,-0.5
3,3,0,f1_micro,0.728241,prototree-h2,random-taxonomy_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,1,random-taxonomy,-0.5
4,4,0,f1_macro,0.742776,prototree-h2,random-taxonomy_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,1,random-taxonomy,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,695,99,hlca-mistake,2.826087,protonet,origin-taxonomy_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,origin-taxonomy,1.0
16796,696,99,f1_micro,0.527778,prototree-h2,origin-taxonomy_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,origin-taxonomy,1.0
16797,697,99,f1_macro,0.514985,prototree-h2,origin-taxonomy_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,origin-taxonomy,1.0
16798,698,99,f1_micro,0.610648,prototree-h1,origin-taxonomy_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,origin-taxonomy,1.0


# Results Table

In [3]:
import numpy as np
metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = list(reversed(df['n_shot'].unique()))
models = list(reversed(df['model'].unique()))

results = {}
for tag in tags:
    results[tag] = []
    for n_shot in shots:
        for model in models:
            if model == 'hproto_v2': continue
                
            ms = {
                    'model': model,
                    'n_shot': n_shot, 
            }
            for metric in metrics:
                values = np.array(df[(df['metric'] == metric) & \
                            (df['tag'] == tag) & \
                            (df['n_shot'] == n_shot) & \
                            (df['model'] == model) 
                           ]['value'].values)
                
                ms[metric] = f'{values.mean():.4f} ± {values.std():.4f}'
            
            results[tag].append(ms)
            

print(f"{pd.DataFrame(results['protonet'], index=None).to_markdown()}")

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered

|    | model              |   n_shot | f1_micro        | f1_macro        | hlca-mistake    |
|---:|:-------------------|---------:|:----------------|:----------------|:----------------|
|  0 | origin-taxonomy_v0 |        1 | 0.5840 ± 0.0764 | 0.5640 ± 0.0786 | 2.5935 ± 0.2410 |
|  1 | dmdb-baseline_v0   |        1 | 0.5773 ± 0.0825 | 0.5560 ± 0.0854 | 2.4606 ± 0.1984 |
|  2 | dmdb-h2protonet_v1 |        1 | 0.5867 ± 0.0850 | 0.5673 ± 0.0886 | 2.4417 ± 0.1854 |
|  3 | random-taxonomy_v0 |        1 | 0.5330 ± 0.0857 | 0.5112 ± 0.0877 | 2.7133 ± 0.1392 |
|  4 | origin-taxonomy_v0 |        2 | 0.6757 ± 0.0650 | 0.6667 ± 0.0674 | 2.5971 ± 0.2106 |
|  5 | dmdb-baseline_v0   |        2 | 0.6578 ± 0.0685 | 0.6450 ± 0.0720 | 2.4246 ± 0.1816 |
|  6 | dmdb-h2protonet_v1 |        2 | 0.6819 ± 0.0681 | 0.6712 ± 0.0716 | 2.3737 ± 0.1992 |
|  7 | random-taxonomy_v0 |        2 | 0.6265 ± 0.0740 | 0.6158 ± 0.0765 | 2.7257 ± 0.1222 |
|  8 | origin-taxonomy_v0 |        4 | 0.7312 ± 0.0626 | 0.7221 ± 0.06

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.
/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered

# Boxplots and Statistical Significance

In [4]:
from scipy.stats import wilcoxon

def significance(df, pairs):
    
    for pair in list(reversed(pairs)):
        (s1, m1), (s2, m2) = pair

        
        df1 = df[(df['n_shot'] == s1) & (df['model'] == m1)].copy()
        df2 = df[(df['n_shot'] == s2) & (df['model'] == m2)].copy()

        stat, p = wilcoxon(df1['value'].values, df2['value'].values)
        
        print(f'\t{pair} --> p < {p}')

In [5]:
import plotly.express as px
import pandas as pd
from itertools import product

metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = df['n_shot'].unique()
models = df['model'].unique()
print(models)

figs = []
for tag in tags:
    for metric in metrics:
        subdf = df
        subdf = df[(df['metric'] == metric) & (df['tag'] == tag)].copy()
        
        # reverse it
        subdf = subdf.astype({'n_shot': 'str'})
        subdf = subdf.iloc[::-1]
        
        pairs = [((str(s1), m1), (str(s2), m2)) for s1, s2, m1, m2 \
                 in product(shots, shots, models, models) \
                    if s1 == s2 and m1 != m2]
        
        print(f'{tag}-{metric}')
        significance(subdf, pairs)

        if len(subdf) == 0:
            continue
        
        range_y = [0, 1] if 'f1' in metric else None
        
        fig = px.box(subdf, x='n_shot', y='value', color='model',\
                     title=f'{tag}-{metric}', range_y=range_y)
        
        figs.append({
            'tag': tag, 
            'metric': metric, 
            'fig': fig
        })

metrics

['random-taxonomy_v0' 'dmdb-h2protonet_v1' 'dmdb-baseline_v0'
 'origin-taxonomy_v0']
protonet-f1_micro
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-baseline_v0')) --> p < 0.22963889529943515
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 0.42904905305586716
	(('1', 'origin-taxonomy_v0'), ('1', 'random-taxonomy_v0')) --> p < 1.7509792332303966e-05
	(('1', 'dmdb-baseline_v0'), ('1', 'origin-taxonomy_v0')) --> p < 0.22963889529943515
	(('1', 'dmdb-baseline_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 0.196668824313612
	(('1', 'dmdb-baseline_v0'), ('1', 'random-taxonomy_v0')) --> p < 0.00018539915369250807
	(('1', 'dmdb-h2protonet_v1'), ('1', 'origin-taxonomy_v0')) --> p < 0.42904905305586716
	(('1', 'dmdb-h2protonet_v1'), ('1', 'dmdb-baseline_v0')) --> p < 0.196668824313612
	(('1', 'dmdb-h2protonet_v1'), ('1', 'random-taxonomy_v0')) --> p < 1.7107737968484607e-05
	(('1', 'random-taxonomy_v0'), ('1', 'origin-taxonomy_v0')) --> p < 1.7509792332303966e-05
	(('1', 'random-tax

protonet-hlca-mistake
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-baseline_v0')) --> p < 1.0271825119150714e-05
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 4.897915405946526e-07
	(('1', 'origin-taxonomy_v0'), ('1', 'random-taxonomy_v0')) --> p < 8.254719019983319e-05
	(('1', 'dmdb-baseline_v0'), ('1', 'origin-taxonomy_v0')) --> p < 1.0271825119150714e-05
	(('1', 'dmdb-baseline_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 0.13207069537634405
	(('1', 'dmdb-baseline_v0'), ('1', 'random-taxonomy_v0')) --> p < 3.5073868857833576e-14
	(('1', 'dmdb-h2protonet_v1'), ('1', 'origin-taxonomy_v0')) --> p < 4.897915405946526e-07
	(('1', 'dmdb-h2protonet_v1'), ('1', 'dmdb-baseline_v0')) --> p < 0.13207069537634405
	(('1', 'dmdb-h2protonet_v1'), ('1', 'random-taxonomy_v0')) --> p < 1.3800602760081443e-15
	(('1', 'random-taxonomy_v0'), ('1', 'origin-taxonomy_v0')) --> p < 8.254719019983319e-05
	(('1', 'random-taxonomy_v0'), ('1', 'dmdb-baseline_v0')) --> p < 3.5073868857833576e-14

prototree-h2-f1_macro
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-baseline_v0')) --> p < 0.15065555303262182
	(('1', 'origin-taxonomy_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 0.5893232417473824
	(('1', 'origin-taxonomy_v0'), ('1', 'random-taxonomy_v0')) --> p < 7.573451718690408e-05
	(('1', 'dmdb-baseline_v0'), ('1', 'origin-taxonomy_v0')) --> p < 0.15065555303262182
	(('1', 'dmdb-baseline_v0'), ('1', 'dmdb-h2protonet_v1')) --> p < 0.24657218196690034
	(('1', 'dmdb-baseline_v0'), ('1', 'random-taxonomy_v0')) --> p < 0.007472552147645046
	(('1', 'dmdb-h2protonet_v1'), ('1', 'origin-taxonomy_v0')) --> p < 0.5893232417473824
	(('1', 'dmdb-h2protonet_v1'), ('1', 'dmdb-baseline_v0')) --> p < 0.24657218196690034
	(('1', 'dmdb-h2protonet_v1'), ('1', 'random-taxonomy_v0')) --> p < 0.0003678938283442541
	(('1', 'random-taxonomy_v0'), ('1', 'origin-taxonomy_v0')) --> p < 7.573451718690408e-05
	(('1', 'random-taxonomy_v0'), ('1', 'dmdb-baseline_v0')) --> p < 0.007472552147645046
	(('1', 'random-

ValueError: The exact distribution of the Wilcoxon test statistic is not implemented for n=0

In [8]:
figs[0]['fig']

In [9]:
figs[1]['fig']

In [10]:
figs[2]['fig']

In [11]:
figs[3]['fig']

In [7]:
figs[4]['fig']